# utils

> This contains useful functions

In [ ]:
#| hide
%load_ext autoreload
%autoreload 2

In [ ]:
#| default_exp utils

In [ ]:
#| hide
from nbdev.showdoc import *

In [ ]:
#| export
import numpy as np
import pandas as pd
import io

from bokeh.plotting import figure
from bokeh.models.tools import BoxZoomTool
from bokeh.models import HoverTool, NumeralTickFormatter, LabelSet
from bokeh.models.glyphs import Patches
from bokeh.models import (
    CustomJS,
    Range1d,
    ColumnDataSource,
)

from collections import defaultdict
import warnings
import gzip
import urllib.request
import os
import re

In [ ]:
#| export
def is_gzipped_file(file_path):
    try:
        with gzip.open(file_path, 'rb') as f:
            # Attempt to read a small chunk from the file
            f.read(1)
        return True
    except IOError:
        return False
    
def download_file(url, save_path):
    if os.path.exists(save_path):
        print(f"File already exists: {save_path}")
    else:
        urllib.request.urlretrieve(url, save_path)
        print(f"File downloaded and saved: {save_path}")

In [ ]:
# Example usage
file_url = 'https://ftp.ncbi.nlm.nih.gov/refseq/H_sapiens/annotation/GRCh38_latest/refseq_identifiers/GRCh38_latest_genomic.gff.gz'
human_genome_gff = 'GRCh38_latest_genomic.gff.gz'

download_file(file_url, human_genome_gff)
is_gzipped_file(human_genome_gff)

File already exists: GRCh38_latest_genomic.gff.gz


True

In [ ]:
#| export
def extract_attribute(input_str:str, #attribute string to parse
                      attr_name:str, #name of the attribute to extract
                     ) -> str:
    """Extracts the attribute called attr_name from the GFF attributes string"""
    
    pattern = f"[{attr_name[0].lower()}{attr_name[0].upper()}]{attr_name[1:]}=(?P<{attr_name}>[^;]+)"
    match = re.search(pattern, input_str)
    if match:
        return match.groupdict()[attr_name]
    else:
        return None

In [ ]:
input_str = 'ID=cds-ATV02827.1;Parent=gene-SaO11_00001;Dbxref=NCBI_GP:ATV02827.1;Name=ATV02827.1;gbkey=CDS;gene=dnaA;locus_tag=SaO11_00001;product=Chromosomal replication initiator protein DnaA;protein_id=ATV02827.1;transl_table=11'
extract_attribute(input_str,"gene")

'dnaA'

In [ ]:
#| hide
input_str = 'locus_tag=SaO11_00001;product=Chromosomal replication initiator protein DnaA;protein_id=ATV02827.1;transl_table=11'
assert extract_attribute(input_str,"gene") == None

In [ ]:
#| export
def extract_all_attributes(input_str:str)->dict:
    """Extracts all attributes from the GFF attributes column"""
    
    pattern = "(?P<key>\w+[-\w]*)=(?P<value>[^;]+)"
    match = re.findall(pattern, input_str)
    d=defaultdict()
    d.update(match)
    return d

In [ ]:
extract_all_attributes(input_str)

defaultdict(None,
            {'locus_tag': 'SaO11_00001',
             'product': 'Chromosomal replication initiator protein DnaA',
             'protein_id': 'ATV02827.1',
             'transl_table': '11'})

In [ ]:
#| export
def attributes_to_columns(features: pd.DataFrame):
    attr_dicts=features.attributes.apply(extract_all_attributes)
    all_keys=list(set().union(*[d.keys() for d in attr_dicts]))
    
    attr_dict=dict([(k,[d.get(k,None) for d in attr_dicts]) for k in all_keys])
    features=features.copy()
    for k,v in attr_dict.items():
        features[k]=v
    
    features.fillna("")
    return features
    

In [ ]:
#| export
def set_positions(annotation: pd.DataFrame, # an annotation DataFrame extracted from a gff file
                            ):
    """Sets left and right as the position of the feature on the sequence, left is always lower than right.
    start and end represent the begining and end of the feature where start can be greater than end depending on the feature strand.
    """
    annotation=annotation.copy()
    annotation.loc[:, "left"] = annotation[["start"]].values
    annotation.loc[:, "right"] = annotation[["end"]].values
    
    mask = annotation["strand"] == "+"
    annotation.loc[mask, "start"] = annotation.loc[mask, "left"].values
    annotation.loc[mask, "end"] = annotation.loc[mask, "right"].values
    
    mask = annotation["strand"] == "-"
    annotation.loc[mask, "start"] = annotation.loc[mask, "right"].values
    annotation.loc[mask, "end"] = annotation.loc[mask, "left"].values
    
    annotation["middle"] = (annotation.right + annotation.left) / 2
    
    return annotation

In [ ]:
#| export
def default_open_gz(gff_path):
    if is_gzipped_file(gff_path):
        return gzip.open(gff_path,'rt')
    else:
        return open(gff_path,'r')
    

In [ ]:
#| export
default_types=["CDS", "repeat_region", "ncRNA", "rRNA", "tRNA"]
default_attributes=["gene", "locus_tag", "product"]


def parse_gff(gff_path:str, # path to the gff file
              seq_id: str=None, # sequence id (first column of the gff)
              bounds: tuple=None, # (left limit, right limit)
              feature_types: list = None, # list of feature types to extract
             )->pd.DataFrame:
    cwd = os.getcwd()
    
    with default_open_gz(gff_path) as gff_file:
        # Create an in-memory file buffer using the io.StringIO class
        file_buffer = io.StringIO()
        default_seq_id=None
        buffer_empty=True
        for line in gff_file:
            if line[0]=="#":
                continue
            else:
                r=line.split('\t')
                if not seq_id and not default_seq_id:
                    default_seq_id=r[0]
                    seq_id=r[0]
                if r[0]==seq_id:
                    if feature_types==None or r[2] in feature_types:
                        if bounds==None or (int(r[3])<bounds[1] and int(r[4])>bounds[0]):
                            # Write each line to the file buffer
                            file_buffer.write(line)
                            buffer_empty=False
                        
                
        # Reset the file pointer to the beginning of the file buffer
        file_buffer.seek(0)
        if buffer_empty:
            warnings.warn("The annotation DataFrame is empty. Check that the feature_types and seq_id are correct.")
            df=pd.DataFrame(columns=["seq_id", "source","type","start","end","score","strand","phase","attributes"])
        else:
            df=pd.read_csv(file_buffer,sep="\t",header=None)
            df.columns=["seq_id", "source","type","start","end","score","strand","phase","attributes"]
            df=attributes_to_columns(df)
            df=set_positions(df)
     
        return df

In [ ]:
df=parse_gff(human_genome_gff, 
             seq_id="NC_000001.11",
             bounds=(10000,50000))
df.head()

,seq_id,source,type,start,end,score,strand,phase,attributes,pct_identity_gap,...,genome,bit_score,blast_score,gene,mol_type,for_remapping,regulatory_class,left,right,middle
0,NC_000001.11,RefSeq,region,1,248956422,.,+,.,ID=NC_000001.11:1..248956422;Dbxref=taxon:9606...,None,...,chromosome,None,None,None,genomic DNA,None,None,1,248956422,124478211.5
1,NC_000001.11,BestRefSeq,pseudogene,11874,14409,.,+,.,"ID=gene-DDX11L1;Dbxref=GeneID:100287102,HGNC:H...",None,...,None,None,None,DDX11L1,None,None,None,11874,14409,13141.5
2,NC_000001.11,BestRefSeq,transcript,11874,14409,.,+,.,ID=rna-NR_046018.2;Parent=gene-DDX11L1;Dbxref=...,None,...,None,None,None,DDX11L1,None,None,None,11874,14409,13141.5
3,NC_000001.11,BestRefSeq,exon,11874,12227,.,+,.,ID=exon-NR_046018.2-1;Parent=rna-NR_046018.2;D...,None,...,None,None,None,DDX11L1,None,None,None,11874,12227,12050.5
4,NC_000001.11,BestRefSeq,exon,12613,12721,.,+,.,ID=exon-NR_046018.2-2;Parent=rna-NR_046018.2;D...,None,...,None,None,None,DDX11L1,None,None,None,12613,12721,12667.0


In [ ]:
#| export
def available_feature_types(gff_path):
    ftypes=set()
    with default_open_gz(gff_path) as handle:
        for line in handle:
            if line[0]!="#":
                r=line.split('\t')
                if len(r)==9:
                    ftypes.add(r[2])
    return ftypes

In [ ]:
from genomenotebook.data import get_example_data_dir
import os

In [ ]:
data_path = get_example_data_dir()
gff_path = os.path.join(data_path, "MG1655_U00096.gff3")
available_feature_types(gff_path)

{'CDS',
 'exon',
 'gene',
 'mobile_genetic_element',
 'ncRNA',
 'origin_of_replication',
 'pseudogene',
 'rRNA',
 'recombination_feature',
 'region',
 'repeat_region',
 'sequence_feature',
 'tRNA'}

In [ ]:
#| export
def available_attributes(gff_path):
    features=parse_gff(gff_path)
    return features.columns

In [ ]:
available_attributes(gff_path)

Index(['seq_id', 'source', 'type', 'start', 'end', 'score', 'strand', 'phase',
       'attributes', 'part', 'locus_tag', 'exception', 'Is_circular', 'gbkey',
       'rpt_type', 'strain', 'gene_biotype', 'protein_id', 'Dbxref',
       'transl_except', 'mobile_element_type', 'recombination_class', 'pseudo',
       'gene_synonym', 'Note', 'orig_transcript_id', 'product',
       'orig_protein_id', 'substrain', 'Name', 'ID', 'Parent', 'genome',
       'gene', 'mol_type', 'transl_table', 'left', 'right', 'middle'],
      dtype='object')

In [ ]:
#| hide
#Testing mistake in seq_id
features=parse_gff(gff_path, "U00097.3") #mistake in seq_id

/tmp/ipykernel_1523/3857466861.py:37: UserWarning: The annotation DataFrame is empty. Check that the feature_types and seq_id are correct.
  warnings.warn("The annotation DataFrame is empty. Check that the feature_types and seq_id are correct.")


In [ ]:
#| export
from collections import defaultdict

In [ ]:
#| export
def get_glyph_definitions():
    glyph_definitions={}
    glyph_definitions["basic_arrow"]={"gylph_type":"arrow",
                                     "colors":("purple","orange"),
                                     "show_name":True}
    glyph_definitions["basic_box"]={"gylph_type":"box",
                                   "colors":("grey",),
                                   "show_name":False}
    return glyph_definitions

def get_default_glyphs():
    """"Returns a dictionnary with:
            - keys: feature types
            - values: a dictionnary with:
                -keys: values: 
                    -gylph_type: arrow or box
                    -colors: a tuple with the colors used for each strand
                    -show_name: a boolean
    """
    glyph_definitions=get_glyph_definitions()
    basic_arrow=glyph_definitions["basic_arrow"]
    basic_box=glyph_definitions["basic_box"]
    default_glyphs=defaultdict(lambda: basic_arrow.copy()) #the default glyph will be the same as for CDS etc.
    default_glyphs.update(dict([(f,basic_arrow.copy()) for f in ["CDS", "ncRNA", "rRNA", "tRNA"]]))
    default_glyphs['repeat_region']=basic_box.copy()
    default_glyphs['exon']=basic_box.copy()
    return default_glyphs

default_glyphs=get_default_glyphs()

In [ ]:
get_default_glyphs()

defaultdict(<function __main__.get_default_glyphs.<locals>.<lambda>()>,
            {'CDS': {'gylph_type': 'arrow',
              'colors': ('purple', 'orange'),
              'show_name': True},
             'ncRNA': {'gylph_type': 'arrow',
              'colors': ('purple', 'orange'),
              'show_name': True},
             'rRNA': {'gylph_type': 'arrow',
              'colors': ('purple', 'orange'),
              'show_name': True},
             'tRNA': {'gylph_type': 'arrow',
              'colors': ('purple', 'orange'),
              'show_name': True},
             'repeat_region': {'gylph_type': 'box',
              'colors': ('grey',),
              'show_name': False},
             'exon': {'gylph_type': 'box',
              'colors': ('grey',),
              'show_name': False}})

In [ ]:
#| export
Y_RANGE = (0, 1)
def get_y_range() -> tuple:
    """Accessor that returns the Y range for the genome browser plot
    """
    return Y_RANGE

gene_y_range = (0.05, 0.2)
def get_gene_y_range() -> tuple:
    """Accessor that returns the Y range for the genome browser plot
    """
    return gene_y_range

In [ ]:
#| export


def arrow_coordinates(feature):
    feature_size = feature.right - feature.left
    
    if feature.strand=="+":
        arrow_base = feature.end - np.minimum(feature_size, 100)
    else:
        arrow_base = feature.end + np.minimum(feature_size, 100)
    
    xs=(feature.start,
        feature.start,
        arrow_base,
        feature.end,
        arrow_base
       )
    
    y_min, y_max = gene_y_range
    ys = (y_min, y_max, y_max, (y_max + y_min) / 2, y_min)
    return xs, ys

def box_coordinates(feature):
    xs=(feature.left, feature.left,
        feature.right, feature.right)
    y_min, y_max = gene_y_range
    ys = (y_min+0.1, y_max-0.1, y_max-0.1, y_min+0.1)
    return xs, ys


def get_patch_coordinates(feature, # row of a pandas DataFrame extracted from a GFF file
                          patch_dict: dict = default_glyphs # a dictionnary containing as key a feature type and as value a patch definition.
                         ):
    """
    patchs are defined with a patch a tuple: (patch_type, (patch_color_plus, patch_color_minus)). 
    Different colors can be specified depending on the strand."""
    
    patch_colors=patch_dict[feature.type]["colors"]
    if len(patch_colors)>1:
        color_dic={"+":patch_colors[0],
                   "-":patch_colors[1]}
    else:
        color_dic=defaultdict(lambda: patch_colors[0])
        
    if patch_dict[feature.type]["gylph_type"]=="arrow":
        return arrow_coordinates(feature), color_dic[feature.strand]
    elif patch_dict[feature.type]["gylph_type"]=="box":
        return box_coordinates(feature), color_dic[feature.strand]
    
    

In [ ]:
features=parse_gff(gff_path, 
                   feature_types=default_types, 
                   seq_id="U00096.3")
coordinates, colors = zip(*features.apply(get_patch_coordinates,axis=1))
coordinates[:5], colors[:5]

((((190, 190, 190, 255, 190), (0.05, 0.2, 0.2, 0.125, 0.05)),
  ((337, 337, 2699, 2799, 2699), (0.05, 0.2, 0.2, 0.125, 0.05)),
  ((2801, 2801, 3633, 3733, 3633), (0.05, 0.2, 0.2, 0.125, 0.05)),
  ((3734, 3734, 4920, 5020, 4920), (0.05, 0.2, 0.2, 0.125, 0.05)),
  ((5234, 5234, 5430, 5530, 5430), (0.05, 0.2, 0.2, 0.125, 0.05))),
 ('purple', 'purple', 'purple', 'purple', 'purple'))

In [ ]:
#| export
def split_string(string, max_length=10):
    if len(string) <= max_length:
        return string
    else:
        split_index = max_length
        while split_index > 0 and string[split_index] != ' ':
            split_index -= 1
        if split_index == 0:
            split_index = max_length  # If no suitable breaking point found, split at max_length
        return string[:split_index] + '\n' + split_string(string[split_index:].lstrip(), max_length)



In [ ]:
# Example usage
long_string = "This is a very long string that needs to be split into multiple lines because it exceeds 50 characters."

split_result = split_string(long_string, max_length=50)
print(split_result)


This is a very long string that needs to be split
into multiple lines because it exceeds 50
characters.


In [ ]:
#| export
def get_feature_name(f,
                     name="gene",
                     patch_dict=default_glyphs,
                    attributes: list = default_attributes, #list of attributes to display when hovering
                    ):
    if patch_dict[f["type"]]["show_name"]:
        if hasattr(f,name):
            if f[name]:
                return f[name]
        
        for attr in attributes:
            if f[attr]:
                return f[attr]
        
        return f[9]
    else:
        return ""

In [ ]:
features.apply(get_feature_name,name="gene",patch_dict=default_glyphs, axis=1)

0       thrL
1       thrA
2       thrB
3       thrC
4       yaaX
        ... 
5251    creC
5252    creD
5253    arcA
5254    yjjY
5255    yjtD
Length: 5256, dtype: object

In [ ]:
#| export
def get_feature_patches(features: pd.DataFrame, #DataFrame of the features 
                        left: int, #left limit
                        right: int, #right limit
                        patch_dict: dict = default_glyphs, #glyphs to use for each feature type
                        attributes: list = default_attributes, #list of attributes to display when hovering
                        name: str = default_attributes[0] #attribute to be displayed as the feature name
                       )->pd.DataFrame:
    features=features.loc[(features["right"] > left) & (features["left"] < right)]
    
    if len(features)>0:
        coordinates, colors = zip(*features.apply(get_patch_coordinates,patch_dict=patch_dict,axis=1))
        xs, ys = zip(*coordinates)
    else:
        colors = []
        xs, ys = [], []
        
    names=list(features.apply(get_feature_name,
                         name=name,
                         patch_dict=default_glyphs,
                         axis=1))
        
    out=dict(names=list(names),
             xs=list(xs),
             ys=list(ys),
             color=list(colors),
             pos=list(features.middle.values),
            )
    for attr in attributes:
        if attr in features.columns:
            values=features[attr].fillna("").astype(str)
            out[attr]=values.to_list() #tried to split long strings here but Bokeh then ignores it 
            
    return pd.DataFrame(out)

 

In [ ]:
get_feature_patches(features,10000,11000, patch_dict=default_glyphs, name="gene")

,names,xs,ys,color,pos,gene,locus_tag,product
0,satP,"(10494, 10494, 10028, 9928, 10028)","(0.05, 0.2, 0.2, 0.125, 0.05)",orange,10211.0,satP,b0010,acetate/succinate:H(+) symporter
1,yaaW,"(11356, 11356, 10743, 10643, 10743)","(0.05, 0.2, 0.2, 0.125, 0.05)",orange,10999.5,yaaW,b0011,putative enzyme-specific chaperone YaaW
2,mbiA,"(10830, 10830, 11215, 11315, 11215)","(0.05, 0.2, 0.2, 0.125, 0.05)",purple,11072.5,mbiA,b0012,uncharacterized protein MbiA


In [ ]:
a=get_feature_patches(features,10000,11000, patch_dict=default_glyphs, name="gene")
pd.DataFrame(a).to_dict(orient="list")==a

,names,xs,ys,color,pos,gene,locus_tag,product
0,False,False,False,False,False,False,False,False
1,False,False,False,False,False,False,False,False
2,False,False,False,False,False,False,False,False


In [ ]:
a

,names,xs,ys,color,pos,gene,locus_tag,product
0,satP,"(10494, 10494, 10028, 9928, 10028)","(0.05, 0.2, 0.2, 0.125, 0.05)",orange,10211.0,satP,b0010,acetate/succinate:H(+) symporter
1,yaaW,"(11356, 11356, 10743, 10643, 10743)","(0.05, 0.2, 0.2, 0.125, 0.05)",orange,10999.5,yaaW,b0011,putative enzyme-specific chaperone YaaW
2,mbiA,"(10830, 10830, 11215, 11315, 11215)","(0.05, 0.2, 0.2, 0.125, 0.05)",purple,11072.5,mbiA,b0012,uncharacterized protein MbiA


In [ ]:
#df=get_feature_patches(features,10000,16000, patch_dict=default_glyphs, name="gene")
#df.to_dict(orient='list')

In [ ]:
#| export
def create_genome_browser_plot(glyphSource, x_range, attributes=default_attributes, **kwargs):
    plot_height = kwargs.get("plot_height", 150)
    label_angle = kwargs.get("label_angle", 45)
    text_font_size = kwargs.get("text_font_size", "10pt")
    output_backend = kwargs.get("output_backend", "webgl")
    
    y_min, y_max = get_y_range()
    p_annot = figure(
        tools = "xwheel_zoom,xpan,save",
        active_scroll = "xwheel_zoom",
        height = plot_height,
        x_range = x_range,
        y_range = Range1d(y_min, y_max),
        output_backend=output_backend,
    )
    # Add tool
    p_annot.add_tools(BoxZoomTool(dimensions="width"))

    # Format x axis values
    p_annot.xaxis[0].formatter = NumeralTickFormatter(format="0,0")
    # Hide grid
    p_annot.xgrid.visible = False
    p_annot.ygrid.visible = False
    # Hide axis
    p_annot.yaxis.visible = False
    glyph = p_annot.add_glyph(
        glyphSource, Patches(xs="xs", ys="ys", fill_color="color")
    )
    # gene labels in the annotation track
    # This seems to be necessary to show the labels
    p_annot.scatter(x="pos", y=0, size=0, source=glyphSource)
    labels = LabelSet(
        x="pos",
        y=gene_y_range[1]+0.02,
        text="names",
        level="glyph",
        angle=label_angle,
        text_font_size=text_font_size,
        x_offset=-5,
        y_offset=0,
        source=glyphSource,
        text_align='left',
    )

    p_annot.add_layout(labels)
    tooltips=[(attr,f"@{attr}") for attr in attributes]
    p_annot.add_tools(
        HoverTool(
            renderers=[glyph],
            tooltips=tooltips,
        )
    )
    return p_annot

In [ ]:
from bokeh.models import (
    Range1d,
    ColumnDataSource,
)
from bokeh.plotting import show

In [ ]:
features = parse_gff(gff_path,
                     feature_types=default_types,
                    )
x_range = Range1d(
            2000, 5000, 
            bounds=(0,5000), 
            max_interval=100000,
            min_interval=40
        )

feature_patches = get_feature_patches(features, x_range.start, x_range.end)
glyph_source = ColumnDataSource(feature_patches)
p = create_genome_browser_plot(glyph_source, x_range)
show(p)

In [ ]:
#| hide
def sort_list_dict(d: dict, #a dictionnary for which all values are of type list
                   ref_list="xs", #key of the list to use as a reference for sorting
                   func=lambda x: x, #A custom function can be supplied to customize the sort order. Default is the identity function.
                  ):
    ks=list(d.keys())
    ref_list_ix=ks.index(ref_list)
    # Sort all the lists in the dictionary based on the values of the reference list
    sorted_lists = sorted(zip(*[d[k] for k in ks]), key= lambda x: func(x[ref_list_ix]))

    # Convert the sorted tuples back into separate lists
    unzipped_lists = zip(*sorted_lists)

    # Create a new dictionary with the same keys as the original dictionary, but with the sorted lists as values
    d = {k: list(t) for k, t in zip(ks, unzipped_lists)}
    return d


In [ ]:
#| hide
d={'xs':[[2,5],[3,1]],'a':[1,3],'b':["c","d"]}
sort_list_dict(d, ref_list='xs', func= lambda x: x[1]) #sort according to the second element of each element of list 'xs'

{'xs': [[3, 1], [2, 5]], 'a': [3, 1], 'b': ['d', 'c']}

In [ ]:
#| hide
import nbdev; nbdev.nbdev_export()